In [37]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

%matplotlib inline

In [38]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [39]:

X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [40]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test_processed = fill_missing_values(X_test)
print(X_train.shape)
print(X_test_processed.shape)

<ipython-input-38-df279fcc57ad>:6: RuntimeWarning: invalid value encountered in true_divide
  X_norma = (X-X_ave)/X_std
<ipython-input-38-df279fcc57ad>:6: RuntimeWarning: invalid value encountered in true_divide
  X_norma = (X-X_ave)/X_std


(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [41]:
X_processed,y_processed = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1200, 828)


## 3. Feature Selection

In [42]:
# pearsonr feature selection
from scipy.stats import pearsonr
from sklearn.decomposition import PCA

def pearsonr_feature_selection(X, y, X_test,degree,weights):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlations = {}
#     print("feature_idx: ", feature_idx.T)
    for idx in feature_idx:
        correlation = 0
        for degree_idx in range(0,len(weights)):
            correlation += abs(weights[degree_idx]*pearsonr(X[:,idx],y**degree[degree_idx])[0])
        correlations[idx] = correlation
    data_correlations = pd.DataFrame(correlations, index=['correlation']).T
    indices_desc = data_correlations['correlation'].abs().sort_values(ascending=False).index
#     print(indices_desc)
    return data_correlations,indices_desc

def pearsonr_feature_selection_correlation(X, X_test,threshold):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlation_select = np.array([True for i in range(0,X.shape[1])])
    for f_idx in feature_idx:
        if(correlation_select[f_idx]):
            feature_co,indices_desc = pearsonr_feature_selection(X[:,(f_idx+1):], X[:,f_idx], X_test_processed,np.array([1]),np.array([1]))
            reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
            remove_index = reomove_features.index+(f_idx+1)
            correlation_select[remove_index] = False
    
    return X[:,correlation_select],X_test[:,correlation_select]

In [67]:
data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.95,0.025,0.025]))

In [68]:
selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.001]
X_selection = X_processed[:,selected_features.index]
X_test_selection = X_test_processed[:,selected_features.index]
print(X_selection.shape)
print(X_test_selection.shape)

(1200, 818)
(776, 818)


<ipython-input-68-db213c50d0b9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.001]


In [143]:
# X, X_test = feature_reduction(X, X_test,750)
X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 54)
print("Traing data shape after selection: {}".format(X_selection_tree.shape))
print("Testing data shape after selection: {}".format(X_test_selection_tree.shape))

Traing data shape after selection: (1200, 54)
Testing data shape after selection: (776, 54)


In [144]:
X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.96)
# X_selection_tree_cor, X_test_selection_tree_cor = X_selection_tree, X_test_selection_tree
print(X_selection_tree_cor.shape)
print(X_test_selection_tree_cor.shape)

<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(featur

(1200, 50)
(776, 50)


<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
<ipython-input-42-4efe4188ad20>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reomove_features = feature_co.loc[indices_desc][abs(featur

## 4. Gaussian Process

In [147]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, DotProduct, WhiteKernel

def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
    kernel = RBF(0.5, (1e-4, 10))
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=1, alpha=0.07, normalize_y=True)
    gpr.fit(X_train, y_train)
    score = gpr.score(X_val, y_val)   
    y_pred = gpr.predict(X_test) 

    return score, y_pred

def train_k_fold(X, y, fold_num=10):
    kf = KFold(n_splits=fold_num, shuffle=True)
    kf.get_n_splits(X)
    test_score = 0.0
    train_score = 0.0
    i = 0
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_val)
        test_score += score
        print(i+1, 'th training. the obtained validation r2 score is : ',score)
        
        score, _ = fit_model_and_pred(X_train, y_train, X_train, y_train, X_val)
        train_score += score
        print(i+1, 'th training. the obtained training r2 score is : ',score)
        print(' ')
        i += 1
       
    print("Validation score: %f"%(test_score/fold_num))
    print("Training score: %f"%(train_score/fold_num))
    
def train_k_fold_predict(X, y, X_test, fold_num=10):
    kf = KFold(n_splits=fold_num)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    cnt = 0
    val_score = 0.0
    train_score = 0.0
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test) 
        val_score += score
        y_test_predict += y_pred
        
        score, _ = fit_model_and_pred(X_train, y_train, X_train, y_train, X_test)
        train_score += score
    return train_score/fold_num, val_score/fold_num, y_test_predict/fold_num

In [152]:
train_score, val_score, y_pred = train_k_fold_predict(X_selection_tree_cor,y_processed,X_test_selection_tree_cor, fold_num=10)
print("Validation score: %f"%(val_score))
print("Training score: %f"%(train_score))

Validation score: 0.669079
Training score: 0.979669


In [153]:
final_res = np.vstack((indices_test, y_pred)).T
df_res = pd.DataFrame(final_res)
df_res.to_csv("our_result.csv", header = ["id", "y"], index=False)

In [155]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 54-50 global maxima

1 th training. the obtained validation r2 score is :  0.7925598135663375
1 th training. the obtained training r2 score is :  0.9801008624286771
 
2 th training. the obtained validation r2 score is :  0.7778186439034196
2 th training. the obtained training r2 score is :  0.9800023178796643
 
3 th training. the obtained validation r2 score is :  0.5625695934229109
3 th training. the obtained training r2 score is :  0.9795756844740485
 
4 th training. the obtained validation r2 score is :  0.6868620277928594
4 th training. the obtained training r2 score is :  0.9793085520049787
 
5 th training. the obtained validation r2 score is :  0.6572446608776558
5 th training. the obtained training r2 score is :  0.9799675748974821
 
6 th training. the obtained validation r2 score is :  0.6501526535907476
6 th training. the obtained training r2 score is :  0.9797603843788393
 
7 th training. the obtained validation r2 score is :  0.6098870358039422
7 th training. the obtained training r2 score is : 

In [105]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 51-48

1 th training. the obtained validation r2 score is :  0.5974452367894867
1 th training. the obtained training r2 score is :  0.9796988505029343
 
2 th training. the obtained validation r2 score is :  0.6859228066827172
2 th training. the obtained training r2 score is :  0.9800567301117994
 
3 th training. the obtained validation r2 score is :  0.6765747979358501
3 th training. the obtained training r2 score is :  0.9795855880594633
 
4 th training. the obtained validation r2 score is :  0.6092886291041888
4 th training. the obtained training r2 score is :  0.98164441077974
 
5 th training. the obtained validation r2 score is :  0.7149490723970613
5 th training. the obtained training r2 score is :  0.980559463031077
 
6 th training. the obtained validation r2 score is :  0.6668026113798982
6 th training. the obtained training r2 score is :  0.9801374116845202
 
7 th training. the obtained validation r2 score is :  0.7149562910976037
7 th training. the obtained training r2 score is :  0.

In [101]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 52-49

1 th training. the obtained validation r2 score is :  0.696307345807096
1 th training. the obtained training r2 score is :  0.9799695069284378
 
2 th training. the obtained validation r2 score is :  0.7521240760757179
2 th training. the obtained training r2 score is :  0.9799150624279189
 
3 th training. the obtained validation r2 score is :  0.6345008385378395
3 th training. the obtained training r2 score is :  0.9804853545593014
 
4 th training. the obtained validation r2 score is :  0.5753154930391655
4 th training. the obtained training r2 score is :  0.9794626665457008
 
5 th training. the obtained validation r2 score is :  0.668150414222155
5 th training. the obtained training r2 score is :  0.9802222972766924
 
6 th training. the obtained validation r2 score is :  0.5858121120031496
6 th training. the obtained training r2 score is :  0.9799541171636451
 
7 th training. the obtained validation r2 score is :  0.7050097134155844
7 th training. the obtained training r2 score is :  0

In [96]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 54-50

1 th training. the obtained validation r2 score is :  0.6016527246615753
1 th training. the obtained training r2 score is :  0.9803988906521187
 
2 th training. the obtained validation r2 score is :  0.6118291410041448
2 th training. the obtained training r2 score is :  0.9789521747593397
 
3 th training. the obtained validation r2 score is :  0.6734784382834074
3 th training. the obtained training r2 score is :  0.9792919557783526
 
4 th training. the obtained validation r2 score is :  0.6876694836724524
4 th training. the obtained training r2 score is :  0.9796187511347894
 
5 th training. the obtained validation r2 score is :  0.7209353767498261
5 th training. the obtained training r2 score is :  0.9795116465134451
 
6 th training. the obtained validation r2 score is :  0.7319464966379295
6 th training. the obtained training r2 score is :  0.9797876341745378
 
7 th training. the obtained validation r2 score is :  0.6063543393785025
7 th training. the obtained training r2 score is : 

In [93]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 55-51

1 th training. the obtained validation r2 score is :  0.7342637745078253
1 th training. the obtained training r2 score is :  0.9802376109769322
 
2 th training. the obtained validation r2 score is :  0.7643200070094
2 th training. the obtained training r2 score is :  0.980527431379008
 
3 th training. the obtained validation r2 score is :  0.7072989141463432
3 th training. the obtained training r2 score is :  0.9798144652547306
 
4 th training. the obtained validation r2 score is :  0.5224585088698427
4 th training. the obtained training r2 score is :  0.9797893393307562
 
5 th training. the obtained validation r2 score is :  0.6503068484280556
5 th training. the obtained training r2 score is :  0.9798684546687546
 
6 th training. the obtained validation r2 score is :  0.6561087032686026
6 th training. the obtained training r2 score is :  0.9797115040586865
 
7 th training. the obtained validation r2 score is :  0.6637041380368374
7 th training. the obtained training r2 score is :  0.9

In [88]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 50-47

1 th training. the obtained validation r2 score is :  0.5580034828122171
1 th training. the obtained training r2 score is :  0.9801107522726294
 
2 th training. the obtained validation r2 score is :  0.6900292669166593
2 th training. the obtained training r2 score is :  0.9805734778628282
 
3 th training. the obtained validation r2 score is :  0.6528167435206538
3 th training. the obtained training r2 score is :  0.9788123553053266
 
4 th training. the obtained validation r2 score is :  0.686843122009343
4 th training. the obtained training r2 score is :  0.9800561432810854
 
5 th training. the obtained validation r2 score is :  0.6117976878445891
5 th training. the obtained training r2 score is :  0.9798380943353041
 
6 th training. the obtained validation r2 score is :  0.6703648127176762
6 th training. the obtained training r2 score is :  0.9789919916840989
 
7 th training. the obtained validation r2 score is :  0.6705593240241539
7 th training. the obtained training r2 score is :  

In [128]:
train_k_fold(X_selection_tree_cor,y_processed, fold_num=10) # 40-39

1 th training. the obtained validation r2 score is :  0.5808155078431745
1 th training. the obtained training r2 score is :  0.9787473116727731
 
2 th training. the obtained validation r2 score is :  0.6293274128391748
2 th training. the obtained training r2 score is :  0.9795814970887748
 
3 th training. the obtained validation r2 score is :  0.619069441339543
3 th training. the obtained training r2 score is :  0.9789401008923845
 
4 th training. the obtained validation r2 score is :  0.5115037037712877
4 th training. the obtained training r2 score is :  0.9798867654143804
 
5 th training. the obtained validation r2 score is :  0.6402413436789616
5 th training. the obtained training r2 score is :  0.9794919099480219
 
6 th training. the obtained validation r2 score is :  0.721281383892558
6 th training. the obtained training r2 score is :  0.9794261430410826
 
7 th training. the obtained validation r2 score is :  0.6824027926996841
7 th training. the obtained training r2 score is :  0